In [ ]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from xgboost import XGBClassifier
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import copy
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import sys

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !cp -r --recursive "/content/glove.6B.zip"     "/content/drive/My Drive/DataScienceCompetitions/glove_embeddings"
!cp -r --recursive  "/content/drive/My Drive/DataScienceCompetitions/glove_embeddings"   "/content/glove.6B.zip"

In [ ]:
# !unzip /content/glove.6B.zip

In [ ]:
!cp -r --recursive "/content/drive/My Drive/DataScienceCompetitions/AV_Independence_day/train.csv"  '/content/train.csv'
!cp -r --recursive "/content/drive/My Drive/DataScienceCompetitions/AV_Independence_day/test.csv"  '/content/test.csv'
train = pd.read_csv('/content/train.csv')
test  = pd.read_csv("/content/test.csv")

In [ ]:
train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [ ]:
print(train["Computer Science"].nunique())
print(train.Physics.nunique())
print(train.Mathematics.nunique())
print(train.Statistics.nunique())
print(train["Quantitative Biology"].nunique())
print(train["Quantitative Finance"].nunique())

2
2
2
2
2
2


In [ ]:
print(train["Computer Science"].value_counts())
print(train["Physics"].value_counts())
print(train["Mathematics"].value_counts())
print(train["Statistics"].value_counts())
print(train["Quantitative Biology"].value_counts())
print(train["Quantitative Finance"].value_counts())

0    12378
1     8594
Name: Computer Science, dtype: int64
0    14959
1     6013
Name: Physics, dtype: int64
0    15354
1     5618
Name: Mathematics, dtype: int64
0    15766
1     5206
Name: Statistics, dtype: int64
0    20385
1      587
Name: Quantitative Biology, dtype: int64
0    20723
1      249
Name: Quantitative Finance, dtype: int64


In [207]:
df_train = pd.read_csv('/content/train.csv')
df_train['train_flag'] = 1
print(df_train.shape)


df_test = pd.read_csv('/content/test.csv')
df_test['train_flag'] = 0
df_test['Computer Science'] =    -1
df_test['Physics'] =             -1
df_test['Mathematics'] =         -1
df_test['Statistics'] =          -1
df_test['Quantitative Biology'] =-1
df_test['Quantitative Finance'] =-1

print(df_test.shape)

df_data = pd.concat([df_train,df_test])
print(df_data.shape)
df_data = df_data.reset_index(drop=True)
df_data.head()

(20972, 10)
(8989, 10)
(29961, 10)


,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,train_flag
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,1
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,1
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,1
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,1
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,1


In [208]:
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [228]:
# here u will get an array with various strings seperated by space
tkr = RegexpTokenizer('[a-zA-Z]+')
train_title = []
train_abs = []
computer = []
physics = []
maths = []
stats = []
bio = []
finance = []

test_title = []
test_abs = []
colID = []

def getTitle(title):
    title = str(title).lower()
    title = tkr.tokenize(title)
    title = " ".join(title)
    return title

def getAbstract(abstract):
    abstract = str(abstract).lower()
    abstract = tkr.tokenize(abstract)
    abstract = " ".join(abstract)
    return abstract

for i in range(len(df_data)):
    abstract = df_data["ABSTRACT"][i]
    abstract = getAbstract(abstract)

    title = df_data["TITLE"][i]
    title = getTitle(title)



    if df_data["train_flag"][i] == 1:
        train_title.append(title)
        train_abs.append(abstract)

        computer.append(df_data["Computer Science"][i])
        physics.append(df_data["Physics"][i])
        maths.append(df_data["Mathematics"][i])
        stats.append(df_data["Statistics"][i])
        bio.append(df_data["Quantitative Biology"][i])
        finance.append(df_data["Quantitative Finance"][i])
    else:
        test_title.append(title)
        test_abs.append(abstract)
        colID.append(df_data["ID"][i])
        

print(df_train.loc[[3]]["TITLE"].values)
print(df_train.loc[[3]]["ABSTRACT"].values)

print(train_title[3])
print(train_abs[3])
print(len(train_abs))


print(df_test.loc[[3]]["TITLE"].values)
print(df_test.loc[[3]]["ABSTRACT"].values)

print(test_title[3])
print(test_abs[3])
print(len(test_abs))

# print(computer[:10])
# print(physics[:10])
# print(maths[:10])
# print(stats[:10])
# print(bio[:10])
# print(finance[:10])

['A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system']
['  The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate solutions, with no or\nminor conditions on time and space steps (depending on the value of $\\theta$).\nHence, we prove the existence of weak martingale solutions of the stochastic\nMLLG system. Numerical results are presented to show applicability of the\nmethod.\n']
a finite element approximation for the stochastic maxwell landau lifshitz gilbert system
the stochastic

In [ ]:
computer = np.array(computer)
maths = np.array(maths)
physics = np.array(physics)
stats = np.array(stats)
bio = np.array(bio)
finance = np.array(finance)



In [240]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(len(stop_words) , stop_words)

for i in range(97,97+26):
    stop_words.append(chr(i))
print(len(stop_words) , stop_words)

stop_words = list(set(stop_words))
print(len(stop_words) , stop_words)


# stop_words = []
# print(len(stop_words) , stop_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
179 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',

In [241]:
arr1 = []
for ind in train_abs:
    ind = ind.split()
    arr = []
    for word in ind:
        if word not in stop_words: 
            arr.append(word)
    sent1 = " ".join(arr)
    arr1.append(sent1)


arr2 = []
for ind in test_abs:
    ind = ind.split()
    arr = []
    for word in ind:
        if word not in stop_words: 
            arr.append(word)
    sent2 = " ".join(arr)
    arr2.append(sent2)


print(df_train.loc[[3]]["TITLE"].values)
print(df_train.loc[[3]]["ABSTRACT"].values)

print(train_title[3])
print(arr1[3])
print(len(train_title))


print(df_test.loc[[3]]["TITLE"].values)
print(df_test.loc[[3]]["ABSTRACT"].values)

print(test_title[3])
print(arr2[3])
print(len(test_title))

['A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system']
['  The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate solutions, with no or\nminor conditions on time and space steps (depending on the value of $\\theta$).\nHence, we prove the existence of weak martingale solutions of the stochastic\nMLLG system. Numerical results are presented to show applicability of the\nmethod.\n']
a finite element approximation for the stochastic maxwell landau lifshitz gilbert system
stochastic lan

In [242]:
train_pr = []
for ind in range(len(train_title)):
    st = train_title[ind] + " " + arr1[ind]
    train_pr.append(st)

test_pr = []
for ind in range(len(test_title)):
    st = test_title[ind] + " " + arr2[ind]
    test_pr.append(st)



print(df_train.loc[[3]]["TITLE"].values)
print(df_train.loc[[3]]["ABSTRACT"].values)
print(train_pr[3])

print(df_test.loc[[3]]["TITLE"].values)
print(df_test.loc[[3]]["ABSTRACT"].values)
print(test_pr[3])


['A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system']
['  The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate solutions, with no or\nminor conditions on time and space steps (depending on the value of $\\theta$).\nHence, we prove the existence of weak martingale solutions of the stochastic\nMLLG system. Numerical results are presented to show applicability of the\nmethod.\n']
a finite element approximation for the stochastic maxwell landau lifshitz gilbert system stochastic lan

In [276]:
# ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams
# max_features len of vocabulary
# fit_transform will be used with sentences used to generate vocab and returns train_array
# transform is used to get test_array using the vocab generated above

from sklearn.feature_extraction.text import CountVectorizer

countvector = CountVectorizer(ngram_range=(1,1), max_features=11000 )
trainX = countvector.fit_transform(train_pr)
testX = countvector.transform(test_pr)

print(len(countvector.get_feature_names()) )
print(countvector.get_feature_names())
print(trainX.shape , testX.shape)
print()

# print(trainX.toarray())
# print(testX.toarray())

11000
['aa', 'ab', 'abc', 'abelian', 'abell', 'aberrations', 'abilities', 'ability', 'ablation', 'able', 'abnormal', 'about', 'abridged', 'abrupt', 'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorber', 'absorbing', 'absorption', 'abstract', 'abstraction', 'abstractions', 'abstracts', 'abundance', 'abundances', 'abundant', 'ac', 'academia', 'academic', 'accelerate', 'accelerated', 'accelerates', 'accelerating', 'acceleration', 'accelerations', 'accelerator', 'accelerators', 'accelerometer', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'acceptor', 'accepts', 'access', 'accessed', 'accesses', 'accessibility', 'accessible', 'accessing', 'accident', 'accidental', 'accidents', 'accommodate', 'accompanied', 'accompanying', 'accomplish', 'accomplished', 'accordance', 'according', 'accordingly', 'account', 'accounted', 'accounting', 'accounts', 'accreted', 'accreting', 'accretion', 'accumulate', 'accumulated', 'accumulation', 'accuracies', 'accuracy'

In [277]:
from sklearn.feature_extraction.text import TfidfTransformer


transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
transformer.fit(trainX)
trainX = transformer.transform(trainX)



transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
transformer.fit(testX)
testX = transformer.transform(testX)



In [278]:
from sklearn.multiclass import OneVsRestClassifier

lr = LogisticRegression(class_weight='balanced',C=3.0)
clf = OneVsRestClassifier(lr)
clf.fit(trainX, np.array([computer,physics,maths,stats,bio,finance]).T)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


OneVsRestClassifier(estimator=LogisticRegression(C=3.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [279]:
preds = clf.predict(testX)

dic = {'ID': colID, 'Computer Science': preds.T[0] , 'Physics':preds.T[1], 'Mathematics':preds.T[2], 'Statistics':preds.T[3], 'Quantitative Biology':preds.T[4], 'Quantitative Finance':preds.T[5]} 
df_fin = pd.DataFrame(dic) 
filename = "sub.csv"
df_fin.to_csv(filename , index=False)
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
.8207   5000
.8265   10000
.8272   11000
.8273   12000
.8266   13000
.8261   15000


In [264]:
# modelC = MultinomialNB()
# modelC.partial_fit(trainX, computer, classes=[0, 1])

# modelC = LogisticRegression(class_weight='balanced',C=3.0).fit(trainX, computer)

# modelC = LinearSVC(class_weight='balanced').fit(trainX,computer)

# modelC = KNeighborsClassifier(n_neighbors=3).fit(trainX, computer)

# modelC = SVC(gamma='auto').fit(trainX, computer)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
.78 multinavebayes    word_vector / count_vector
.8039 multinavebayes    tfidf

.8157 logistic regre    tfidf

.7935 KNN tfidf

worst svm